
# Prática: AlexNet com recursos limitados

Se antes implementamos a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) sem considerar a questão de quantidade de parâmetros, nesta prática focaremos nesse quesito.






In [1]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary


import time, os, sys, numpy as np

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))
print(device)

cuda


In [2]:
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Cifar10-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.CIFAR10(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.CIFAR10(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## AlexNet

Como vimos, a rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) foi uma das arquiteturas mais famosas dessa nova onda de rede neurais.

<p align="center">
  <img width=700 src="https://miro.medium.com/max/700/1*vXBvV_Unz3JAxytc5iSeoQ.png">
</p>

Entretanto, ela possui muitos parâmetros.
Especificamente, essa arquitetura, para classificar 10 classes, tem um total de **58.312.736** de parâmetros como mostrado na tabela abaixo.

**Camada** | **Calc Parâmetros** | **Total Parâmetros**
--- | ---: | ---:
Convolução 1 | 11\*11\*3\*96 | 34.848
Convolução 2 | 5\*5\*96\*256 | 614.400
Convolução 3 | 3\*3\*256\*384 | 884.736
Convolução 4 | 3\*3\*384\*384 | 1.327.104
Convolução 5 | 3\*3\*384\*256 | 884.736
FC 6 | 9216*4096 | 37.748.736
FC 7 | 4096*4096 | 16.777.216
FC 8 | 4096*10 | 40.960
**Total** | | **58.312.736**

**Seu objetivo nessa prática é propor uma nova rede neural, baseada na [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), que possuia MENOS parâmetros e alcance uma acurácia similar ou melhor que a rede original vista na aula passada.**

Procure usar [*batch normalization*](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html), camadas dilatadas e separáveis.
Neste caso, desconsidere os parâmetros da camada [*batch normalization*](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html).

### Rede original para efeito de comparação

In [15]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),

            nn.Linear(9216, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [16]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

net = AlexNet(3, 10)

# Sending model to device
net.to(device)
print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Flatten-14                 [-

### Arquitetura 1

Essa primeira versão da [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) usa [*batch normalization*](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) para melhorar o resultado. Esse arquitetura é somente para efeitos de comparação já que, por enquanto, ainda temos a mesma quantidade de parâmetros da versão original.

In [18]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=96),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=384),

            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=384),

            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),

            nn.Linear(9216, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=4096),
            nn.Dropout(0.5),

            nn.Linear(4096, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [19]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

net = AlexNet(3, 10)

# Sending model to device
net.to(device)
print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
       BatchNorm2d-3           [-1, 96, 55, 55]             192
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
       BatchNorm2d-7          [-1, 256, 27, 27]             512
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
      BatchNorm2d-11          [-1, 384, 13, 13]             768
           Conv2d-12          [-1, 384, 13, 13]       1,327,488
             ReLU-13          [-1, 384, 13, 13]               0
      BatchNorm2d-14          [-1, 384,

### Arquitetura 2

Essa segunda versão da [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) usa [*batch normalization*](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) e camadas de [convolução dilatada](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.htmll).
Neste caso, como usamos filtros dilatados em duas camadas (com dilatação aumentante, ou seja, dilatação 2 seguida da dilatação 4), foram removidos duas camadas convolucionais, já que o *receptive field* se mantem similar dessa forma.

Nessa arquitetura, já temos menos parâmetros. Precisamente, temos 



In [21]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=96),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2, dilation=2),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            # nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.BatchNorm2d(num_features=384),

            # nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.BatchNorm2d(num_features=384),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, dilation=4),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=256),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),

            nn.Linear(1024, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=4096),
            nn.Dropout(0.5),

            nn.Linear(4096, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [22]:
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

net = AlexNet(3, 10)

# Sending model to device
net.to(device)
print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
       BatchNorm2d-3           [-1, 96, 55, 55]             192
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 23, 23]         614,656
              ReLU-6          [-1, 256, 23, 23]               0
       BatchNorm2d-7          [-1, 256, 23, 23]             512
         MaxPool2d-8          [-1, 256, 11, 11]               0
            Conv2d-9            [-1, 256, 5, 5]         590,080
             ReLU-10            [-1, 256, 5, 5]               0
      BatchNorm2d-11            [-1, 256, 5, 5]             512
        MaxPool2d-12            [-1, 256, 2, 2]               0
          Flatten-13                 [-1, 1024]               0
           Linear-14                 [-